We're working on a test framework to write tests for the below (bad) implementation of `find_twos`. 

In [ ]:
def find_twos(first_list, second_list):
    return []

So far we have written some matchers for our test framework: `has_items`, `has_size`, `is_empty`, and `equals`. For convenience, I stuck our implementations into a file called `matchers` inside a package called `phoenix_test` _right here in this folder_ (you can even go look at it!)

That's right; I have decided that our test framework will be named PhonenixTest. Here is the adorable logo, drawn by yours truly (with a crappy pen on a post-it note, you're welcome to make me a better one and I'll replace this rendition):

![](../images/phoenix_test.jpg)

ANYWAY, this next line imports all that stuff we wrote:

In [ ]:
from phoenix_test.matchers import FailedAssertion, Assertion, assert_that

We can use that stuff to write some tests for our mediocre-at-best implementation of `find_twos` like so:

In [2]:
    def test_empty_inputs():
        print("")
        assert_that(find_twos("", "")).is_empty()
        assert_that(find_twos("2", "")).is_empty()
        assert_that(find_twos("2", "")).is_empty()

    def test_non_matching_sets():
        assert_that(find_twos("1", "1, 3")).is_empty()

    def test_non_matching_twos():
        assert_that(find_twos("2", "1, 3")).is_empty()
        
    def test_matches():
        # This test is going to fail because our find_twos method works crappy
        # This is by design to make sure our testing framework...actually tests something
        assert_that(find_twos("12", "2, 12")).has_size(1)
        assert_that(find_twos("12", "2, 12")).equals([12])
        assert_that(find_twos("1, 2, 20, 22, 44, 99", "3, 5, 22, 100, 44, 2")).has_size(2)
        assert_that(find_twos("1, 2, 20, 22, 44, 99", "3, 5, 22, 100, 44, 2")).has_items(2, 22)

What happens when we run the above block of code?

Ah, yes, nothing. The reason nothing happens is that the above block of code _defines_ four test methods, but it doesn't _call_ them. In order to run these tests, we have to call our test methods like so:

In [ ]:
test_empty_inputs()
test_non_matching_sets()
test_non_matching_twos()
test_matches()

But that's kind of annoying. We don't want to have to explicitly _call_ every test that we write like this. It would be annoying to have to keep a file that just lists out and calls every single test method so we can run our tests all at once...plus, that's mistake-prone, since we might write a test and forget to put it in the list, or delete one and forget to take it out.

What if our `Test` superclass were able to run all our tests for us?

In [ ]:
class Test():
    # Runs all the test methods. HOW?!?!
    def run(self):
        test_methods = [
            token for token in dir(self) \
            if token.startswith("test")  \
            and callable(getattr(self.__class__, token))
        ]
        for method in test_methods:
            print(f"Running {method}.")
            try:
                getattr(self.__class__, method).__call__(self)
            except Exception as e:
                print(e) 

### Let's look at what is happening in this `run` method.

1. "test_methods" is assigned using a **list comprehension**
1. The backslashes in the list comprehension allow me to split what would be a very long line of Python into multiple lines for better legibility.

You should recognize:
1. The for loop
1. The f-string in the print statement

### Challenge: 

What are these things doing?

1. The `dir` method
1. The `.startswith()` method
1. The `.__class__` method
1. The `try` and `except` blocks
1. The `.__getattr__` method
1. The `callable` method
1. The `__call__` method

### This is very important: 

You will spend 90% of your programming time _reading_ code and the other 10% _writing_ code. So it is critical to practice _reading_ code and understanding what it is doing.

### Code Investigation Tool #1: Python's Built-In Documentation

Python provides you with some assistance for researching code that you are reading:

In [ ]:
dir.__doc__

In [ ]:
help(dir)

So the Test class is going to be our **superclass**. We can now **subclass** that Test class like so:

In [ ]:
class FindTwosTest(Test):
    test_useless_attribute = None
    test_other_useless_attribute = None

    def test_empty_inputs(self):
        print("")
        assert_that(find_twos("", "")).equals([])
        assert_that(find_twos("2", "")).equals([])
        assert_that(find_twos("2", "")).equals([])

    def test_non_matching_sets(self):
        assert_that(find_twos("1", "1, 3")).equals([])

    def test_non_matching_twos(self):
        assert_that(find_twos("2", "1, 3")).equals([])
        
    def test_matches(self):
        assert_that(find_twos("12", "2, 12")).equals([12])
        assert_that(find_twos("1, 2, 20, 22, 44, 99", "3, 5, 22, 100, 44, 2")).equals([2, 22])

In [ ]:
FindTwosTest().run()

### Code Investigation Tool #2: Running Your Own Experiments

On _this particular_ code, I have kept it in small chunks inside of a REPL environment so that you can remove or change lines of code to investigate what they are doing.

### Challenge: 

1. What happens if you remove the "f" from the front of the f-string?
1. What happens if you comment out `and callable(getattr(self.__class__, token))` in the list comprehension?
1. What happens if you remove the try/except above and just call `getattr(self.__class__, method).__call__(self)` right after the print statement?

### We have a test runner. Woo!

Now, things could be better about this test runner. 

### Challenge: 

Get the test output to count up and print out passages and failures for the test class that you've run.

In [ ]:
FindTwosTest().run()

### Challenge:

Get the test output to print in COLORS!

Below see a block of example code to help you get started on that:

In [ ]:
import sys

# For this, we need to install a library.
# When code requires a library to do something, we call that a dependency.
!{sys.executable} -m pip install colorama 

from colorama import Fore, Back, Style 
print(Fore.RED + 'some red text') 
print(Fore.GREEN + 'and some green text') 
print(Back.YELLOW + 'you can also do backgrounds')
print(Style.RESET_ALL) 
print('back to normal now')

In [ ]:
FindTwosTest().run()

## Great Job!

Next week, we will make our test framework even more sophisticated. For example, we might address these questions:
    
- What if I have several different subclasses of `Test` and I want to run _all_ my tests at once?
- What if I have some code that I need to run before and after every test?
- What if I want to be able to skip a test, or only run specific tests according to some rule I made up?

Think about how you might do these things, because lots of programmers want to use your framework and they're blowing up your phone and emailing you day and night asking for new features!